In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from numpy import cos
from math import log
from scipy.interpolate import interp1d
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize

#### As equações do modelo:

$dShdt = -a  b_2  (I_M / N)  S_H \\
dIhdt = a  b_2  (I_M / N)  S_H - \gamma  I_H \\
dRhdt = \gamma  I_H$

$dSmdt = b - a  b_1  (I_H / N)  S_M - \mu S_M \\
dEmdt = a b_1  (I_H / N)  S_M - \mu E_M - b_3 E_M l \\
dImdt = b_3  E_M  l - \mu I_M$

In [2]:
%display typeset

### Analisando o SIR:

In [3]:
var('S_H I_H R_H a b2 I_M N gamma')

(S_H, I_H, R_H, a, b2, I_M, N, gamma)

In [4]:
dShdt = -a * b2 * (I_M / N) * S_H
dIhdt = a * b2 * (I_M / N) * S_H - gamma * I_H
dRhdt = gamma * I_H

Dado que o compartimento dos Recuperados é desacoplado da dinâmica, podemos ignorá-lo na análise.

#### Calculando o equilíbrio do SIR

In [5]:
solve([dShdt,dIhdt, dRhdt],[S_H,I_H, R_H])

[[S_H == 0, I_H == 0, R_H == r1]]

#### Calculando a Jacobiana do SIR

In [6]:
jack_sir=jacobian([dShdt,dIhdt, dRhdt],[S_H,I_H, R_H])
jack_sir

[-I_M*a*b2/N           0           0]
[ I_M*a*b2/N      -gamma           0]
[          0       gamma           0]

In [7]:
cp_sir = jack_sir.characteristic_polynomial()
cp_sir

x^3 + (I_M*a*b2/N + gamma)*x^2 + I_M*a*b2*gamma/N*x

In [8]:
jack_sir.eigenvalues()

[-I_M*a*b2/N, -gamma, 0]

### Analisando o SEI:

In [9]:
var('S_M E_M I_M b a b1 I_H N mu b3 l')

(S_M, E_M, I_M, b, a, b1, I_H, N, mu, b3, l)

In [10]:
dSmdt = b - a * b1 * (I_H / N) * S_M - mu * S_M
dEmdt = a * b1 * (I_H / N) * S_M - mu * E_M - b3 * E_M * l
dImdt = b3 * E_M * l - mu * I_M

#### Calculando o equilíbrio do SEI

In [11]:
solve([dSmdt,dEmdt,dIhdt],[S_M,E_M,I_M])

[[S_M == N*b/(I_H*a*b1 + N*mu), E_M == I_H*a*b*b1/(I_H*a*b1*b3*l + N*mu^2 + (I_H*a*b1 + N*b3*l)*mu), I_M == I_H*N*gamma/(S_H*a*b2)]]

#### Calculando a Jacobiana do SEI

In [12]:
jack_sei=jacobian([dSmdt,dEmdt,dIhdt],[S_M,E_M,I_M])
jack_sei

[-I_H*a*b1/N - mu                0                0]
[      I_H*a*b1/N       -b3*l - mu                0]
[               0                0       S_H*a*b2/N]

In [13]:
cp_sei = jack_sei.characteristic_polynomial()
cp_sei

x^3 + (I_H*a*b1/N - S_H*a*b2/N + b3*l + 2*mu)*x^2 + (-I_H*S_H*a^2*b1*b2/N^2 + I_H*a*b1*b3*l/N - S_H*a*b2*b3*l/N + I_H*a*b1*mu/N - 2*S_H*a*b2*mu/N + b3*l*mu + mu^2)*x - I_H*S_H*a^2*b1*b2*b3*l/N^2 - I_H*S_H*a^2*b1*b2*mu/N^2 - S_H*a*b2*b3*l*mu/N - S_H*a*b2*mu^2/N

In [14]:
jack_sei.eigenvalues()

[-(I_H*a*b1 + N*mu)/N, S_H*a*b2/N, -b3*l - mu]

### ----------------------------------------
### ----------------------------------------

### Calculando $\mathcal{R}_0$:

Usando o método proposto por P. van den Driessche no seguinte artigo, e indicado pelo professor Flavio na aula 9 de Modelagem de Fenômenos Biológicos: 

[Reproduction numbers and sub-threshold endemic equilibria for compartmental models of disease transmission](https://pdfs.semanticscholar.org/3cf7/1968a86800215b4e129ec3eda67520832cf0.pdf)

[Aula 9](https://github.com/fccoelho/Modelagem-Matematica-IV/blob/master/Planilhas%20Sage/Aula%209%20-%20Calculo%20do%20R0.ipynb)


Seja $X=(x_1,\ldots, x_n)^T$, tal que $x_i\geq 0$ seja o número de indivíduos em cada compartimento.

Definindo $X_s$ como o conjunto de todos os estados livres de doença.

$$X_s=\{x \geq 0|x_i=0, i=1,\ldots,m\}$$

Supõe-se que cada função é continuamente diferenciável pelo menos duas vezes ($C^2$) em cada variável. As equações são reordenadas para que as $m$ primeiras equações sejam as que contém infectados. Seja ${\cal F}_i(x)$ a taxa de aparecimento de novas infecções no compartimento $i$, ${\cal V}_i^+(x)$ a taxa de entrada de indivíduos no compartimento $i$ por outros meios e ${\cal V}_i^-(x)$ a taxa de saída de indivíduos do compartimento $i$. O modelo de transmissão da doença consiste em condições iniciais não negativas juntamente com o seguinte sistema de equações:

$\dot{x}=f_i(x)={\cal F}_i(x)-{\cal V}_i(x), i=1,\ldots, n$

Onde ${\cal V}_i (x) = {\cal V}_i^{-}(x) - {\cal V}_i^+(x)$ e as funções satisfazem os pressupostos  (A1) - (A5) descritos abaixo. Desde que cada função representa uma transferência dirigida de indivíduos, todos elas são não-negativas.

(A1) Se $x \geq 0 $, então ${\cal F}_i, {\cal V}_i^+, {\cal V}_i ^- \geq 0$ para $i=1, \ldots, n$,

ou seja, se um compartimento estiver vazio, não pode haver saída de indivíduos deste, por morte, infecção ou qualquer outro meio.

(A2) Se $x_i=0$ então ${\cal V}_i^-(x)=0$. Em particular, se $x \in X_s$, então ${\cal V}_i^-(x)=0$ para $i=1,\ldots, m$

(A3) ${\cal F}_i=0$ se $i>m$

(A4) Se $x \in X_S$, então ${\cal F}_i(x) = 0$ e ${\cal V}_i^+(x)=0$ para $i=1,\ldots, m$

(A5) Se ${\cal F}(x)$ é um vetor nulo, então todos os autovalores de $Df(x_0)$ tem parte real negativa.

### ----------------------------------------

Para calcular o $R_0$ é importante distinguir as novas infecções de todas as outras mudanças na população. Nos modelos propostos, os compartimentos que correspondem aos indivíduos infectados são $I$, no caso do SIR, e $E$ e $I$ no caso do SEI. Vamos analisá-los separadamente: 

#### SIR

${\bf m=1}$. A fim de clareza, vamos ordenar os $n=2$ compartimentos da seguinte forma: $[I, S]$, separando os $m$ primeiros compartimentos do restante. Podemos também normalizar, de forma a tirar o denominador $N$.

$$ {\cal F}_i(x): \text{ taxa de surgimento de novos infectados no compartimento } i $$


$$ {\cal F} =\begin{bmatrix}
a  b_2  I_M  S_H \\
\end{bmatrix} $$

In [15]:
F_cal_sir = matrix([[a*b2*I_M*S_H]])
F_cal_sir

[I_M*S_H*a*b2]

Além disso, temos

$$ {\cal V}_i(x)^-: \text{ taxa de saída do compartimento } i $$

$$ {\cal V}_i(x)^+: \text{ taxa de entrada do compartimento } i $$

Logo:

$$
\begin{equation}
{\cal V^-} = \begin{bmatrix}
\gamma I_H\\
\end{bmatrix}
\end{equation}
$$

$$
\begin{equation}
{\cal V^+} = \begin{bmatrix}
0\\
\end{bmatrix}
\end{equation}
$$

In [16]:
V_cal_neg_sir = matrix([[gamma*I_H]])
V_cal_neg_sir

[I_H*gamma]

In [17]:
V_cal_pos_sir = matrix([[0]])
V_cal_pos_sir

[0]

$${\cal V}_i (x) = {\cal V}_i(x)^{-} - {\cal V}_i(x)^+$$

Então,
\begin{equation}
{\cal V} =
\begin{bmatrix}
\gamma I_H\\
\end{bmatrix}
\end{equation}

In [18]:
V_cal_sir = V_cal_neg_sir-V_cal_pos_sir
V_cal_sir

[I_H*gamma]

$$ F = \dfrac{\partial{\cal F}}{\partial I_M} =\begin{bmatrix}
a  b_2  S_H \\
\end{bmatrix} $$

In [19]:
F_sir = jacobian(F_cal_sir(S_H=1),[I_M])
F_sir

[a*b2]

$$ V = \dfrac{\partial{\cal V}}{\partial I_H} =\begin{bmatrix}
\gamma \\
\end{bmatrix} $$

In [20]:
V_sir = jacobian(V_cal_sir(S_H=1),[I_H])
V_sir

[gamma]

${\cal R}_0 = \rho (FV^{-1})$

In [21]:
M_sir = F_sir*V_sir.inverse()
M_sir 

[a*b2/gamma]

In [22]:
M_sir=M_sir.simplify_full()
M_sir

[a*b2/gamma]

In [23]:
M_sir.eigenvalues()

[a*b2/gamma]

In [24]:
R0_sir=M_sir[0,0].simplify_full()
R0_sir

a*b2/gamma

In [25]:
R0_sir=max(M_sir.eigenvalues())
R0_sir

a*b2/gamma

In [26]:
R0_sir.variables()

(a, b2, gamma)

In [27]:
Ft_sir = jacobian(F_cal_sir,[I_H])
Vt_sir = jacobian(V_cal_sir,[I_H])
Mt_sir = Ft_sir*Vt_sir.inverse()
show(pretty_print(html('$R_t=$')))
Rt_sir = Mt_sir[0,0]
Rt_sir

<script type="math/tex">R_t=</script>

None

0

In [28]:
print(Rt_sir)

0


#### SEI

${\bf m=2}$. A fim de clareza, vamos ordenar os $n=3$ compartimentos da seguinte forma: $[E, I, S]$, separando os $m$ primeiros compartimentos do restante. Vale ressaltar que as transferências dos compartimentos expostos para os infectados não são consideradas novas infecções, mas sim a progressão de um indivíduo infectado através dos vários compartimentos. Novamente removendo $N$, portanto,

$$ {\cal F}_i(x): \text{ taxa de surgimento de novos infectados no compartimento } i $$


$$ {\cal F} =\begin{bmatrix}
a b_1 I_H S_M\\
0\\
\end{bmatrix} $$

In [29]:
F_cal_sei = matrix([[a*b1*I_H*S_M],[0]])
F_cal_sei

[I_H*S_M*a*b1]
[           0]

Além disso, temos

$$ {\cal V}_i(x)^-: \text{ taxa de saída do compartimento } i $$

$$ {\cal V}_i(x)^+: \text{ taxa de entrada do compartimento } i $$

Logo:

$$
\begin{equation}
{\cal V^-} = \begin{bmatrix}
E_M (\mu + b_3 l)\\
\mu I_M
\end{bmatrix}
\end{equation}
$$

$$
\begin{equation}
{\cal V^+} = \begin{bmatrix}
0\\
b_3 E_M l\\
\end{bmatrix}
\end{equation}
$$

In [30]:
V_cal_neg_sei = matrix([[mu * E_M + b3 * E_M * l], [mu * I_M]])
V_cal_neg_sei

[E_M*b3*l + E_M*mu]
[           I_M*mu]

In [31]:
V_cal_pos_sei = matrix([[0],[b3 * E_M * l]])
V_cal_pos_sei

[       0]
[E_M*b3*l]

$${\cal V}_i (x) = {\cal V}_i(x)^{-} - {\cal V}_i(x)^+$$

Então,
\begin{equation}
{\cal V} =
\begin{bmatrix}
E_M (\mu + b_3 l)\\
\mu I_M - b_3 E_M l\\
\end{bmatrix}
\end{equation}

In [32]:
V_cal_sei = V_cal_neg_sei-V_cal_pos_sei
V_cal_sei

[ E_M*b3*l + E_M*mu]
[-E_M*b3*l + I_M*mu]

Definimos também $F=\left[\frac{\partial {\cal F}_i (x_0)}{\partial x_j}\right]$ e $V=\left[\frac{\partial {\cal V}_i (x_0) }{\partial x_j}\right]$, onde $x_0$ é um DFE (Equilíbrio livre de doença) e $1\leq i,j \leq m$. 

Isto equivale à jacobiana  destas duas matrizes, após substituir $x_0$ ou seja, $S=1$.

$$ F = \dfrac{\partial{\cal F}}{\partial E_M, I_H} =\begin{bmatrix}
\dfrac{\partial ab_1 I_H S_M}{\partial E_M} & \dfrac{\partial ab_1 I_H S_M}{\partial I_H}\\
\dfrac{\partial 0}{\partial E_M} & \dfrac{\partial 0}{\partial I_H}\\
\end{bmatrix} = 
\begin{bmatrix}
0 & ab_1 S_M\\
0 & 0\\
\end{bmatrix}$$

In [33]:
F_sei = jacobian(F_cal_sei(S_M=1),[E_M, I_H])
F_sei

[   0 a*b1]
[   0    0]

$$ V = \dfrac{\partial{\cal V}}{\partial E_M, I_M} =\begin{bmatrix}
\dfrac{\partial E_M (\mu + b_3 l)}{\partial E_M} & \dfrac{\partial E_M (\mu + b_3 l)}{\partial I_M}\\
\dfrac{\partial \mu I_M - b_3 E_M l}{\partial E_M} & \dfrac{\partial \mu I_M - b_3 E_M l}{\partial I_M}\\
\end{bmatrix} = 
\begin{bmatrix}
\mu+b_3 l & 0\\
- b_3 l & \mu\\
\end{bmatrix}$$

In [34]:
V_sei = jacobian(V_cal_sei(S_M=1),[E_M, I_M])
V_sei

[b3*l + mu         0]
[    -b3*l        mu]

${\cal R}_0 = \rho (FV^{-1})$

In [35]:
M_sei = F_sei*V_sei.inverse()
M_sei 

[a*b1*b3*l/((b3*l + mu)*mu)                    a*b1/mu]
[                         0                          0]

In [36]:
M_sei =M_sei.simplify_full()
M_sei 

[a*b1*b3*l/(b3*l*mu + mu^2)                    a*b1/mu]
[                         0                          0]

In [37]:
M_sei.eigenvalues()

[a*b1*b3*l/(b3*l*mu + mu^2), 0]

In [38]:
R0_sei=M_sei[0,0].simplify_full()
R0_sei

a*b1*b3*l/(b3*l*mu + mu^2)

In [39]:
R0_sei=max(M_sei.eigenvalues())
R0_sei

a*b1*b3*l/(b3*l*mu + mu^2)

In [40]:
R0_sei.variables()

(a, b1, b3, l, mu)

In [41]:
Ft_sei = jacobian(F_cal_sei,[E_M, I_M])
Vt_sei = jacobian(V_cal_sei,[E_M, I_M])
Mt_sei = Ft_sei*Vt_sei.inverse()
show(pretty_print(html('$R_t=$')))
Rt_sei = Mt_sei[0,0]
Rt_sei

<script type="math/tex">R_t=</script>

None

0

In [42]:
print(Rt_sei)

0


### Modelo acoplado:

${\bf m=3}$. A fim de clareza, vamos ordenar os $n=6$ compartimentos da seguinte forma: $[I_H, E_M, I_M, S_H, S_M, R_M]$, separando os $m$ primeiros compartimentos do restante. Vale ressaltar que as transferências dos compartimentos expostos para os infectados não são consideradas novas infecções, mas sim a progressão de um indivíduo infectado através dos vários compartimentos. Novamente removendo $N$, portanto,

$$ {\cal F}_i(x): \text{ taxa de surgimento de novos infectados no compartimento } i $$


$$ {\cal F} =\begin{bmatrix}
a  b_2  I_M  S_H \\
a b_1 I_H S_M\\
0\\
\end{bmatrix} $$

In [43]:
F_cal = matrix([[a*b2*I_M*S_H],[a*b1*I_H*S_M],[0]])
F_cal

[I_M*S_H*a*b2]
[I_H*S_M*a*b1]
[           0]

Além disso, temos

$$ {\cal V}_i(x)^-: \text{ taxa de saída do compartimento } i $$

$$ {\cal V}_i(x)^+: \text{ taxa de entrada do compartimento } i $$

Logo:

$$
\begin{equation}
{\cal V^-} = \begin{bmatrix}
\gamma I_H\\
E_M (\mu + b_3 l)\\
\mu I_M
\end{bmatrix}
\end{equation}
$$

$$
\begin{equation}
{\cal V^+} = \begin{bmatrix}
0\\
0\\
b_3 E_M l\\
\end{bmatrix}
\end{equation}
$$

In [44]:
V_cal_neg = matrix([[gamma*I_H],[mu * E_M + b3 * E_M * l], [mu * I_M]])
V_cal_neg

[        I_H*gamma]
[E_M*b3*l + E_M*mu]
[           I_M*mu]

In [45]:
V_cal_pos = matrix([[0],[0],[b3 * E_M * l]])
V_cal_pos

[       0]
[       0]
[E_M*b3*l]

$${\cal V}_i (x) = {\cal V}_i(x)^{-} - {\cal V}_i(x)^+$$

Então,
\begin{equation}
{\cal V} =
\begin{bmatrix}
I_H \gamma \\
E_M (\mu + b_3 l)\\
\mu I_M - b_3 E_M l\\
\end{bmatrix}
\end{equation}

In [46]:
V_cal = V_cal_neg-V_cal_pos
V_cal

[         I_H*gamma]
[ E_M*b3*l + E_M*mu]
[-E_M*b3*l + I_M*mu]

Definimos também $F=\left[\frac{\partial {\cal F}_i (x_0)}{\partial x_j}\right]$ e $V=\left[\frac{\partial {\cal V}_i (x_0) }{\partial x_j}\right]$, onde $x_0$ é um DFE (Equilíbrio livre de doença) e $1\leq i,j \leq m$. 

Isto equivale à jacobiana  destas duas matrizes, após substituir $x_0$ ou seja, $S=1$.

$$ F = \dfrac{\partial{\cal F}}{\partial I_H, E_M, I_M} =\begin{bmatrix}
\dfrac{\partial ab_2 I_M S_H}{\partial I_H} & \dfrac{\partial ab_2 I_M S_H}{\partial E_M} & \dfrac{\partial ab_2 I_M S_H}{\partial I_M}\\
\dfrac{\partial ab_1 I_H S_M}{\partial I_H} & \dfrac{\partial ab_1 I_H S_M}{\partial E_M} & \dfrac{\partial ab_1 I_H S_M}{\partial I_M}\\
\dfrac{\partial 0}{\partial I_H} & \dfrac{\partial 0}{\partial E_M} & \dfrac{\partial 0}{\partial I_M}\\
\end{bmatrix} = 
\begin{bmatrix}
0 & 0 & ab_2 S_H\\
ab_1 S_M & 0 & 0\\
0 & 0 & 0
\end{bmatrix}$$

In [47]:
F = jacobian(F_cal(S_H=1, S_M=1),[I_H, E_M, I_M])
F

[   0    0 a*b2]
[a*b1    0    0]
[   0    0    0]

$$ V = \dfrac{\partial{\cal V}}{\partial I_H, E_M, I_M} =\begin{bmatrix}
\dfrac{\partial \gamma I_H}{\partial I_H} & \dfrac{\partial \gamma I_H}{\partial E_M} & \dfrac{\partial \gamma I_H}{\partial I_M}\\
\dfrac{\partial E_M (\mu + b_3 l)}{\partial I_H} & \dfrac{\partial E_M (\mu + b_3 l)}{\partial E_M} & \dfrac{\partial E_M (\mu + b_3 l)}{\partial I_M}\\
\dfrac{\partial \mu I_M - b_3 E_M l}{\partial I_H} & \dfrac{\partial \mu I_M - b_3 E_M l}{\partial E_M} & \dfrac{\partial \mu I_M - b_3 E_M l}{\partial I_M}\\
\end{bmatrix} = 
\begin{bmatrix}
\gamma & 0 & 0\\
0 & b_3l+\mu & 0\\
0 & -b_3l & \mu
\end{bmatrix}$$

In [48]:
V = jacobian(V_cal(S_H=1, S_M=1),[I_H, E_M, I_M])
V

[    gamma         0         0]
[        0 b3*l + mu         0]
[        0     -b3*l        mu]

In [49]:
M = F*V.inverse()
M

[                         0 a*b2*b3*l/((b3*l + mu)*mu)                    a*b2/mu]
[                a*b1/gamma                          0                          0]
[                         0                          0                          0]

In [50]:
M = M.simplify_full()
M

[                         0 a*b2*b3*l/(b3*l*mu + mu^2)                    a*b2/mu]
[                a*b1/gamma                          0                          0]
[                         0                          0                          0]

In [51]:
M.eigenvalues()

[-sqrt(b1*b2*b3*l/(b3*gamma*l*mu + gamma*mu^2))*a,
 sqrt(b1*b2*b3*l/(b3*gamma*l*mu + gamma*mu^2))*a,
 0]

In [52]:
R0=M[0,0].simplify_full()
R0

0

In [53]:
R0=max(M.eigenvalues())
R0

-sqrt(b1*b2*b3*l/(b3*gamma*l*mu + gamma*mu^2))*a

In [54]:
eigenvalues_list = M.eigenvalues()
eigenvalues_list.sort()

In [55]:
absolute_eigenvalues = np.abs(eigenvalues_list)

# Find the maximum absolute eigenvalue, which is the spectral radius
R0 = np.max(absolute_eigenvalues)
R0

abs(sqrt(b1*b2*b3*l/(b3*gamma*l*mu + gamma*mu^2))*a)